# Schema extraction

## Schema definition



In [1]:
node_schema = {
    'name': '',
    'type': '',
    'values': ['']
}

# conditions and condition_values should be matched
# value should be in the values of the variable
arc_schema = {
    'conditions': [''],
    'condition_values': [''],
    'variable': '',
    'value': '',
    'probability': ''
}

In [1]:
import os

from zhipuai import ZhipuAI
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate

os.environ["ZHIPUAI_API_KEY"] = '8adac289dfd59cb2006ccea31a82efab.GhrrpvmaygKVqjxS'


llm = ChatZhipuAI(
    temperature=0,
    model="glm-4"
)

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

from typing import List


extract_template = """\
For the following text, identify the fundamental goal inplied by the context (which may not directly appear in the text), and extract every variable, action or event. Output using the following format:

name: Extract the core concept or variable\
related to decision-making, and output its name.
    
type: What is the nature of the variable? \
If it is an aleatory variable that cannot be intervened, output 'chance';\
if it can be intervened and represents a decision to be made, output 'decision';\
if it reflects the fundamental goal of decision-maker in the setting, output 'utility'.

values: Extract the possible values of the variable, complete with common knowledge if necessary, and output them as a Python list.

text: {text}

{format_instructions}
"""

class Node(BaseModel):
    variable_name: str = Field(description="Extract the core concept or variable related to decision-making, and output its name.")
    variable_type: str = Field(description="What is the nature of the variable? If it is an aleatory variable that cannot be intervened, output 'chance'; if it can be intervened and represents a decision to be made, output 'decision'; if it reflects the fundamental goal of decision-maker in the setting, output 'utility'.")
    values: list = Field(description="Extract the possible values of the variable, complete with common knowledge if necessary, and output them as a Python list.")

class List_of_Nodes(BaseModel):
    node_list: List[Node]

parser = JsonOutputParser(pydantic_object=List_of_Nodes)

prompt = PromptTemplate(
    template=extract_template,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

text = """Openings connecting upper and
lower floors within buildings compromise the in-
tegrity of fire compartments, potentially leading to
the spread of fire across multiple areas and floors.
Therefore, reliable fire separation measures should
be implemented in these connected spaces to pre-
vent the rapid upward spread of fire."""

contents = chain.invoke({"text": text})

In [3]:
contents['node_list']

[{'variable_name': 'Fire_Separation_Measures',
  'variable_type': 'decision',
  'values': ['Implement', 'Not_Implement']},
 {'variable_name': 'Fire_Spread',
  'variable_type': 'chance',
  'values': ['Rapid_Upward', 'Contained']},
 {'variable_name': 'Fire_Integrity_Compromise',
  'variable_type': 'chance',
  'values': ['Compromised', 'Intact']},
 {'variable_name': 'Utility',
  'variable_type': 'utility',
  'values': ['Prevent_Rapid_Upward_Spread']}]